In [1]:
from clit_recommender.util import flat_map, iterate_dirs
from clit_recommender import MD_ONLY


files = list(
    flat_map(
        lambda x: iterate_dirs(x, False), flat_map(iterate_dirs, iterate_dirs(MD_ONLY))
    )
)


print(len(files))
print(files[0])

19464
/Users/niladi/workspace/seminar-kdd/data/2024-05-14/MDOnly/KORE_50_DBpedia.ttl/TextRazor/a7542e21d3d55116ca5c88cc0660f6347e54a74a244856ad7951ee5f5b15e984.json


In [2]:
from clit_recommender.clit_result import ClitResult

with open(files[0], "r") as f:
    j = f.read()
    clit_result = ClitResult.from_json(j)

In [5]:
task = clit_result.experiment_tasks[0]
task.pipeline_config

PipelineConfig(start_components=['MD1'], components={'md': [{'id': 'MD1', 'value': 'TextRazor'}], 'cg_ed': []}, example_id='md_combined_cged', end_components=['MD1'], display_name='MD + combined CG-ED', id=1, connections=[], pipeline_config_type='complex')

In [11]:
from pynif import NIFCollection
from clit_recommender import DATASETS_PATH, MD_ONLY
from os.path import isdir
from rdflib import Graph, Namespace
from os import listdir
from clit_recommender.clit_result import ClitResult, Document, ExperimentTask, Mention
from clit_recommender.util import flat_map

namespace = Namespace("http://aifb.kit.edu/clit/recommender")


for dataset in listdir(DATASETS_PATH):
    dataset_path = f"{DATASETS_PATH}/{dataset}"
    g = Graph()
    g.bind("aifb", namespace)
    try:
        g.parse(dataset_path, format="ttl")
        print("Dataset:", dataset)
    except:
        print("Can't Parse dataset", dataset)
        continue
    for system in listdir(f"{MD_ONLY}/{dataset}"):
        if not isdir(f"{MD_ONLY}/{dataset}/{system}"):
            continue
        print("System:", system)
        for experiment in listdir(f"{MD_ONLY}/{dataset}/{system}"):
            clit_result: ClitResult
            try:
                with open(f"{MD_ONLY}/{dataset}/{system}/{experiment}", "r") as f:
                    j = f.read()
                    clit_result = ClitResult.from_json(j)
            except:
                print("Can't Parse experiment", experiment)
                continue
            task: ExperimentTask
            for task in clit_result.experiment_tasks:
                document: Document
                for document in flat_map(lambda x: x, task.documents):
                    text = document.text

                    query = f"""
                        SELECT ?context
                        WHERE {{
                            ?context rdf:type nif:Context .
                            ?context nif:isString "{text}" .
                        }}
                    """
                    print(query)
                    res = g.query(query)
                    for r in res:
                        print(r)
                    assert len(res) == 1
                    print(res)
                    exit()
                    mention: Mention
                    for mention in document.mentions:
                        mention.mention

Can't Parse dataset .DS_Store
Dataset: KORE_50_DBpedia.ttl
System: TextRazor

                        SELECT ?context
                        WHERE {
                            
                            ?context nif:isString "Haug congratulated Red Bull." .
                        }
                    


AssertionError: 